In [2]:
import numpy as np
import matplotlib.pyplot as plt
# from cellpose import core, utils, io, models, metrics
import os
import time
import tkinter as tk
import re
import datetime
from tifffile import imread, imwrite
from tqdm import tqdm

In [3]:
dir = '/media/rodriguez/DVP/onur/20240125_NG_555cfos_647neun_tiffs/'
os.chdir(dir)

In [4]:
os.getcwd()

'/media/rodriguez/DVP/onur/20240125_NG_555cfos_647neun_tiffs'

In [5]:
# Instantiate folders
folders = ['image_input', 'image_output', 'mask_output']
for folder in folders:
    if not os.path.exists(folder):
        os.mkdir(folder)

In [6]:
def list_files(dir_location: str,
               file_extension: str,
               include_names=None,
               exclude_names=None):
    '''
    Returns a list of file names, and paths where each file
    matches all the criteria.
    '''
    
    list_fnames = os.listdir(dir_location)

    list_fnames = [x for x in list_fnames if x.endswith(file_extension)]

    # Files must include all of the include_names
    if include_names is not None:
        for include_name in include_names:
            list_fnames = [x for x in list_fnames if include_name in x]
            # print(f'Finding `{include_name}`, found {len(list_fnames)}')

    # Files must exclude any of the exclude_names
    if exclude_names is not None:
        for exclude_name in exclude_names:
            list_fnames = [x for x in list_fnames if exclude_name not in x]

    list_fpath = [os.path.join(dir_location, x) for x in list_fnames]

    return list_fpath

In [7]:
lst_images_in = list_files('image_input/', 'tiff', ['c1'])
print("number of tiles:", len(lst_images_in))

number of tiles: 604


In [9]:
# Define the number of channels and the names of the fluorescent channels
num_channels = 3
# channels_chosen = [1,2,3]
# channel_names = ['SYTOG', 'AF555', 'AF647']
channels_chosen = [2,1,3]
channel_names = ['AF555', 'SYTOG','AF647']
#order of channel names here will show up in cellpose as R, g, b

channels = [None] * num_channels
for i, ch in enumerate(channels_chosen):
    channels[i] = [x.replace('_c1_', f'_c{ch}_') for x in lst_images_in]

lst_images_out = [x.replace('input', 'output') for x in lst_images_in]
for idx, image_name in enumerate(lst_images_in):
    lst_images_out[idx] = re.sub('_c1_', f'_c{num_channels}_', lst_images_out[idx], 1)

for idx, image_name in tqdm(enumerate(lst_images_in)):
    # print(f'Image {idx}, file name: {image_name}')

    images = [None] * num_channels
    for i in range(num_channels):
        images[i] = imread(channels[i][idx])

    comb = np.dstack(images)

    # print(f' |-> Outfile image: {lst_images_out[idx]}')
    imwrite(lst_images_out[idx], comb)

604it [15:59,  1.59s/it]


In [7]:
channel_names

['Alexa647', 'TRITC', 'HOECHST33342']

In [7]:
# define model
model_path = 'Marc_HEK293'
#model_type = "TN2"

# define segmentation channel
cyto_channel = channel_names.index('TRITC') + 1
#nuc_channel = channel_names.index('SytoxGreen') + 1
nuc_channel = channel_names.index('HOECHST33342') + 1

print(cyto_channel, " = membrane channel =", channel_names[cyto_channel-1],"\n",nuc_channel, " = nuclear channel =",  channel_names[nuc_channel-1])

2  = membrane channel = TRITC 
 3  = nuclear channel = HOECHST33342


In [8]:
from threading import Thread
from time import sleep
import sys

def timer(prognosed_time):
    global status
    prognosed_time = prognosed_time * 60
    start_time = time.time()
    while True:
        elapsed_time = time.time() - start_time
        remaining_time = prognosed_time - elapsed_time
        if status == "completed":
            print("Segmentation completed!")
            return
        print("Elapsed Time:", np.round(elapsed_time/60, 2), "min. Remaining Time:", np.round(remaining_time/60, 2), end='\r')
        sleep(10)      
        
def calculate_cellpose():
    
    global status, images,lst_images_out, channel_names, model_path, diameter, flow_threshold, cellprob_threshold, cyto_channel, nuc_channel
    with open("log.txt", "w") as f:
    
        start = time.process_time()

        print("Image:", os.path.basename(os.getcwd()), file=f)
        print("Current date and time:", datetime.datetime.now().strftime('%d.%m.%Y, %H:%M:%S'), file=f)
        print("Number of tiles segmented:", len(lst_images_out), file=f)
        print("Segmenting on: membrane marker", channel_names[cyto_channel-1],"& nuclear marker", channel_names[nuc_channel-1], file=f)    
        print("Model used:", model_path, file=f)
        print(f'Model parameters: {diameter:.3f} diameter, {flow_threshold} flow_threshold, {cellprob_threshold} cellprob_threshold.', file=f)

        masks, flows, styles = model.eval(images, 
                                            channels=[cyto_channel, nuc_channel],
                                            diameter=diameter,
                                            flow_threshold=flow_threshold,
                                            cellprob_threshold=cellprob_threshold)

        end = time.process_time()

        print("Done. process_time()", (end - start) / 60, "min.", "\nNow SAVING MASKS as tiff", file=f)

        # save masks
        lst_mask_out = [x.replace('image_output', 'mask_output') for x in lst_images_out]
        start = time.process_time()

        io.save_masks(images,
                      masks, 
                      flows, 
                      lst_mask_out, 
                      channels=[cyto_channel, nuc_channel],
                      png=False, # save masks as PNGs and save example image
                      tif=True, # save masks as TIFFs
                      save_txt=False, # save txt outlines for ImageJ
                      save_flows=False, # save flows as TIFFs
                      save_outlines=False, # save outlines as TIFFs 
                      )

        mask_files = list_files('mask_output/', 'tif', ['cp_masks'])
        [os.rename(x, x.replace('_cp_masks', '')) for x in mask_files]

        end = time.process_time()

        # print elapsed time in seconds
        print("Done. Process_time()", (end - start) * 10**3, "ms.", file=f)
        
        status = "completed"
        return()
    
# get image files in dir 
images = [io.imread(f) for f in lst_images_out]

# Load model for custom model
model = models.CellposeModel(gpu=True, pretrained_model=model_path)

# Load available model instead of custom model using this command
# model = models.CellposeModel(gpu=True, model_type='TN2')

# Define the parameters of cellpose, DEFINE DIAMETER if not defined by custom model (in this case use 0) !!!

diameter = 0
diameter = model.diam_labels if diameter==0 else diameter
flow_threshold = 0.4
cellprob_threshold  = 0
    
print("Number of tiles to segment:", len(lst_images_out))
print("Segmenting on: membrane marker", channel_names[cyto_channel-1],"& nuclear marker", channel_names[nuc_channel-1])
print("Model used:", model_path)
print(f'Model parameters: {diameter:.3f} diameter, {flow_threshold} flow_threshold, {cellprob_threshold} cellprob_threshold.')    
estimated_time =  (len(lst_images_out) * 0.194)
print(f"Estimated time for segmentation:{estimated_time} min")

status = "not complete"

t1 = Thread(target=timer, args=(estimated_time,))
t2 = Thread(target=calculate_cellpose)
t1.start()
t2.start()

t2.join()  # wait for calculate_cellpose() to complete

print("done")

Number of tiles to segment: 453
Segmenting on: membrane marker TRITC & nuclear marker HOECHST33342
Model used: Marc_HEK293
Model parameters: 35.836 diameter, 0.4 flow_threshold, 0 cellprob_threshold.
Estimated time for segmentation:87.882 min
Segmentation completed!. Remaining Time: 54.02
done


In [16]:
# CLEAN VERSION without 2 threads, just cellpose function
# get image files in dir 
# print(lst_images_out)
images = [io.imread(f) for f in lst_images_out]

# Load model for custom model
model = models.CellposeModel(gpu=True, pretrained_model=model_path)

# Load available model
#model = models.CellposeModel(gpu=True, model_type='TN2')

# Define the parameters of cellpose, DEFINE DIAMETER !!!

diameter = 0
diameter = model.diam_labels if diameter==0 else diameter
flow_threshold = 0.4
cellprob_threshold  = 0
    
print("Number of tiles to segment:", len(lst_images_out))
print("Segmenting on: membrane marker", channel_names[cyto_channel-1],"& nuclear marker", channel_names[nuc_channel-1])
print("Model used:", model_path)
print(f'Model parameters: {diameter:.3f} diameter, {flow_threshold} flow_threshold, {cellprob_threshold} cellprob_threshold.')    
print("Estimated time for segmentation:", (len(lst_images_out) * 0.194), "min")



# start cellpose
with open("log.txt", "w") as f:
    
    start = time.process_time()
    
    print("Image:", os.path.basename(os.getcwd()), file=f)
    print("Current date and time:", datetime.datetime.now().strftime('%d.%m.%Y, %H:%M:%S'), file=f)
    print("Number of tiles segmented:", len(lst_images_out), file=f)
    print("Segmenting on: membrane marker", channel_names[cyto_channel-1],"& nuclear marker", channel_names[nuc_channel-1], file=f)    
    print("Model used:", model_path, file=f)
    print(f'Model parameters: {diameter:.3f} diameter, {flow_threshold} flow_threshold, {cellprob_threshold} cellprob_threshold.', file=f)
    
    masks, flows, styles = model.eval(images, 
                                        channels=[cyto_channel, nuc_channel],
                                        diameter=diameter,
                                        flow_threshold=flow_threshold,
                                        cellprob_threshold=cellprob_threshold)

    end = time.process_time()

    print("Done. process_time()", (end - start) / 60, "min.", "\nNow SAVING MASKS as tiff", file=f)

    # save masks
    lst_mask_out = [x.replace('image_output', 'mask_output') for x in lst_images_out]
    start = time.process_time()

    io.save_masks(images,
                  masks, 
                  flows, 
                  lst_mask_out, 
                  channels=[cyto_channel, nuc_channel],
                  png=False, # save masks as PNGs and save example image
                  tif=True, # save masks as TIFFs
                  save_txt=False, # save txt outlines for ImageJ
                  save_flows=False, # save flows as TIFFs
                  save_outlines=False, # save outlines as TIFFs 
                  )

    mask_files = list_files('mask_output/', 'tif', ['cp_masks'])
    [os.rename(x, x.replace('_cp_masks', '')) for x in mask_files]

    end = time.process_time()

    # print elapsed time in seconds
    print("Done. Process_time()", (end - start) * 10**3, "ms.", file=f)
    
print("Done")

Number of tiles to segment: 11
Segmenting on: membrane marker TRITC & nuclear marker HOECHST33342
Model used: MelDVP_stringentCPbased_H80p07
Model parameters: 80.021 diameter, 0.4 flow_threshold, 0 cellprob_threshold.
Estimated time for segmentation: 2.134 min
Done
